# 📊 Demo Logging Setup

This notebook sets up a structured JSON logging system for the KlarText Demo UI.

**Goals:**
1. Create a logging module that captures each simplification output
2. Store logs in a single JSONL file with running averages
3. Integrate the logger with the demo app

**Log Structure:**
```json
{
    "timestamp": "2026-01-08T14:30:00Z",
    "source_text": "...",
    "output_text": "...",
    "model": "llama-3.1-8b-instant",
    "template": "system_prompt_de.txt",
    "metrics": {
        "avg_sentence_len_words": 8.5,
        "pct_sentences_gt20": 0.0,
        "ari_score": 7.0,
        "meaning_cosine": 0.85
    },
    "guardrails_passed": 4,
    "guardrails_total": 4,
    "guardrails_failed": []
}
```

---
## Step 1: Setup & Imports

In [1]:
import os
import re
import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Project paths
PROJECT_ROOT = Path("..")
DATA_DIR = PROJECT_ROOT / "data"
LOGS_DIR = DATA_DIR / "logs"

# Ensure logs directory exists
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# Log file path
LOG_FILE = LOGS_DIR / "demo_outputs.jsonl"

print(f"✅ Log file will be stored at: {LOG_FILE.resolve()}")

✅ Log file will be stored at: /Users/simonvoegely/Desktop/easysprache/klartext/data/logs/demo_outputs.jsonl


---
## Step 2: Define Metrics Calculation Functions

These functions compute the readability metrics for each output.

In [2]:
def split_sentences(text: str) -> list[str]:
    """
    Split text into sentences.
    Handles common abbreviations in both English and German.
    """
    # Handle common abbreviations
    text_clean = re.sub(r'\b(Mr|Mrs|Ms|Dr|Prof|Jr|Sr|vs|etc|e\.g|i\.e)\.\s', r'\1_DOT ', text)
    # Handle German abbreviations
    text_clean = re.sub(r'\b(z\.B|d\.h|usw|ggfs)\.\s', r'\1_DOT ', text_clean)
    
    # Split on sentence-ending punctuation
    sentences = re.split(r'[.!?]+\s+', text_clean.strip())
    return [s.strip() for s in sentences if s.strip()]


def get_words(text: str) -> list[str]:
    """
    Extract words from text.
    Handles German umlauts and special characters.
    """
    return re.findall(r'[A-Za-zÄÖÜäöüßéèêëàâáîïíôöóûüú]+', text)


def count_syllables(word: str) -> int:
    """
    Estimate syllable count for a word.
    Simple heuristic for English/German.
    """
    word = word.lower()
    vowels = "aeiouyäöü"
    count = 0
    prev_was_vowel = False
    
    for char in word:
        is_vowel = char in vowels
        if is_vowel and not prev_was_vowel:
            count += 1
        prev_was_vowel = is_vowel
    
    # Words always have at least one syllable
    return max(1, count)


def compute_ari_score(text: str) -> float:
    """
    Compute Automated Readability Index (ARI).
    ARI = 4.71 * (characters/words) + 0.5 * (words/sentences) - 21.43
    Lower scores = easier to read.
    """
    sentences = split_sentences(text)
    words = get_words(text)
    
    if not sentences or not words:
        return 0.0
    
    # Count characters (only letters)
    char_count = sum(len(w) for w in words)
    word_count = len(words)
    sentence_count = len(sentences)
    
    ari = 4.71 * (char_count / word_count) + 0.5 * (word_count / sentence_count) - 21.43
    return round(max(0, ari), 2)


def compute_meaning_cosine(source: str, output: str) -> float:
    """
    Compute TF-IDF cosine similarity between source and output.
    Higher scores = better meaning preservation.
    """
    try:
        vectorizer = TfidfVectorizer(lowercase=True)
        tfidf_matrix = vectorizer.fit_transform([source, output])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        return round(similarity, 3)
    except Exception:
        return 0.0


def compute_metrics(source_text: str, output_text: str) -> dict:
    """
    Compute all metrics for a simplification output.
    
    Returns:
        dict with metrics:
        - avg_sentence_len_words: Average words per sentence
        - pct_sentences_gt20: Percentage of sentences > 20 words
        - ari_score: Automated Readability Index
        - meaning_cosine: TF-IDF similarity (meaning preservation)
    """
    sentences = split_sentences(output_text)
    words = get_words(output_text)
    
    if not sentences or not words:
        return {
            "avg_sentence_len_words": 0.0,
            "pct_sentences_gt20": 0.0,
            "ari_score": 0.0,
            "meaning_cosine": 0.0
        }
    
    # Sentence length analysis
    sent_lengths = [len(get_words(s)) for s in sentences]
    avg_sent_len = sum(sent_lengths) / len(sentences)
    long_sents = sum(1 for length in sent_lengths if length > 20)
    pct_long = (long_sents / len(sentences)) * 100
    
    return {
        "avg_sentence_len_words": round(avg_sent_len, 1),
        "pct_sentences_gt20": round(pct_long, 1),
        "ari_score": compute_ari_score(output_text),
        "meaning_cosine": compute_meaning_cosine(source_text, output_text)
    }


print("✅ Metrics functions defined")

✅ Metrics functions defined


---
## Step 3: Define Guardrails Evaluation

Guardrails are binary checks that validate the output quality.

In [3]:
# Guardrails definitions
GUARDRAILS = {
    "short_sentences": {
        "name": "Short Sentences",
        "description": "Average sentence length ≤ 15 words",
        "check": lambda metrics: metrics["avg_sentence_len_words"] <= 15
    },
    "no_long_sentences": {
        "name": "No Long Sentences",
        "description": "Less than 10% of sentences > 20 words",
        "check": lambda metrics: metrics["pct_sentences_gt20"] <= 10
    },
    "readable": {
        "name": "Readable (ARI)",
        "description": "ARI score ≤ 8 (8th grade level or below)",
        "check": lambda metrics: metrics["ari_score"] <= 8
    },
    "preserves_meaning": {
        "name": "Preserves Meaning",
        "description": "Cosine similarity ≥ 0.70 with source",
        "check": lambda metrics: metrics["meaning_cosine"] >= 0.70
    }
}


def evaluate_guardrails(metrics: dict) -> tuple[int, int, list[str]]:
    """
    Evaluate all guardrails against computed metrics.
    
    Returns:
        tuple of (passed_count, total_count, failed_guardrail_names)
    """
    passed = 0
    failed = []
    
    for guardrail_id, guardrail in GUARDRAILS.items():
        try:
            if guardrail["check"](metrics):
                passed += 1
            else:
                failed.append(guardrail["name"])
        except Exception as e:
            failed.append(f"{guardrail['name']} (error)")
    
    return passed, len(GUARDRAILS), failed


print("✅ Guardrails defined")
print(f"   Total guardrails: {len(GUARDRAILS)}")
for gid, g in GUARDRAILS.items():
    print(f"   - {g['name']}: {g['description']}")

✅ Guardrails defined
   Total guardrails: 4
   - Short Sentences: Average sentence length ≤ 15 words
   - No Long Sentences: Less than 10% of sentences > 20 words
   - Readable (ARI): ARI score ≤ 8 (8th grade level or below)
   - Preserves Meaning: Cosine similarity ≥ 0.70 with source


---
## Step 4: Create the Logging Function

This function creates a log entry and appends it to the JSONL file.

In [4]:
def create_log_entry(
    source_text: str,
    output_text: str,
    model: str,
    template: str,
    language: str = "de"
) -> dict:
    """
    Create a complete log entry for a simplification output.
    
    Args:
        source_text: Original input text
        output_text: Simplified output text
        model: Model identifier (e.g., "llama-3.1-8b-instant")
        template: Template filename (e.g., "system_prompt_de.txt")
        language: Language code ("de" or "en")
    
    Returns:
        dict: Complete log entry
    """
    # Get current timestamp in ISO format
    timestamp = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
    
    # Compute metrics
    metrics = compute_metrics(source_text, output_text)
    
    # Evaluate guardrails
    passed, total, failed = evaluate_guardrails(metrics)
    
    return {
        "timestamp": timestamp,
        "source_text": source_text,
        "output_text": output_text,
        "model": model,
        "template": template,
        "language": language,
        "metrics": metrics,
        "guardrails_passed": passed,
        "guardrails_total": total,
        "guardrails_failed": failed
    }


def append_log_entry(log_entry: dict, log_file: Path = LOG_FILE) -> None:
    """
    Append a log entry to the JSONL file.
    
    Args:
        log_entry: Log entry dict
        log_file: Path to the log file
    """
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(log_entry, ensure_ascii=False) + "\n")


def log_simplification(
    source_text: str,
    output_text: str,
    model: str,
    template: str,
    language: str = "de",
    log_file: Path = LOG_FILE
) -> dict:
    """
    Main logging function - creates entry and appends to file.
    
    Returns:
        dict: The created log entry
    """
    entry = create_log_entry(source_text, output_text, model, template, language)
    append_log_entry(entry, log_file)
    return entry


print("✅ Logging functions defined")

✅ Logging functions defined


---
## Step 5: Create Aggregate Statistics Function

This function reads all log entries and computes running averages.

In [5]:
def load_all_logs(log_file: Path = LOG_FILE) -> list[dict]:
    """
    Load all log entries from the JSONL file.
    
    Returns:
        list of log entry dicts
    """
    if not log_file.exists():
        return []
    
    logs = []
    with open(log_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                logs.append(json.loads(line))
    return logs


def compute_aggregate_stats(logs_or_log_file) -> dict:
    """
    Compute aggregate statistics across all log entries.

    Accepts either:
    - a list of log dicts (in-memory), or
    - a log file path (Path/str)

    Returns:
        dict with average metrics and summary stats
    """
    # Allow passing a path (keeps this consistent with demo/demo_logger.py)
    if isinstance(logs_or_log_file, (str, Path)):
        logs = load_all_logs(Path(logs_or_log_file))
    else:
        logs = logs_or_log_file

    if not logs:
        return {
            "total_entries": 0,
            "avg_metrics": {},
            "guardrails_summary": {}
        }

    # Extract metrics
    metrics_list = [log["metrics"] for log in logs if "metrics" in log]

    # Compute averages
    avg_metrics = {}
    if metrics_list:
        metric_keys = metrics_list[0].keys()
        for key in metric_keys:
            values = [m[key] for m in metrics_list if isinstance(m.get(key), (int, float))]
            if values:
                avg_metrics[f"avg_{key}"] = round(sum(values) / len(values), 2)

    # Guardrails summary
    total_passed = sum(log.get("guardrails_passed", 0) for log in logs)
    total_total = sum(log.get("guardrails_total", 0) for log in logs)

    # Count failures by type
    failure_counts = {}
    for log in logs:
        for failed in log.get("guardrails_failed", []):
            failure_counts[failed] = failure_counts.get(failed, 0) + 1

    return {
        "total_entries": len(logs),
        "avg_metrics": avg_metrics,
        "guardrails_summary": {
            "total_passed": total_passed,
            "total_checks": total_total,
            "pass_rate": round(total_passed / total_total * 100, 1) if total_total > 0 else 0,
            "failure_counts": failure_counts
        }
    }


def get_logs_with_summary(log_file: Path = LOG_FILE) -> dict:
    """
    Get all logs with aggregate summary at the top.

    Returns:
        dict with 'summary' and 'entries' keys
    """
    logs = load_all_logs(log_file)
    summary = compute_aggregate_stats(log_file)

    return {
        "summary": summary,
        "entries": logs
    }


def display_summary(log_file: Path = LOG_FILE):
    """
    Display a formatted summary of all logs.
    """
    data = get_logs_with_summary(log_file)
    summary = data["summary"]
    
    print("="*60)
    print("📊 DEMO OUTPUT LOGS - SUMMARY")
    print("="*60)
    print(f"\nTotal entries: {summary['total_entries']}")
    
    if summary["avg_metrics"]:
        print("\n📈 Average Metrics:")
        for key, value in summary["avg_metrics"].items():
            print(f"   {key}: {value}")
    
    if summary["guardrails_summary"]:
        gs = summary["guardrails_summary"]
        print(f"\n✅ Guardrails Pass Rate: {gs['pass_rate']}%")
        print(f"   ({gs['total_passed']}/{gs['total_checks']} checks passed)")
        
        if gs["failure_counts"]:
            print("\n❌ Most Common Failures:")
            sorted_failures = sorted(gs["failure_counts"].items(), key=lambda x: -x[1])
            for name, count in sorted_failures:
                print(f"   - {name}: {count} times")
    
    print("\n" + "="*60)


print("✅ Aggregate statistics functions defined")

✅ Aggregate statistics functions defined


---
## Step 6: Test the Logging System

Let's test with a sample simplification.

In [6]:
# Sample source text (complex)
sample_source = """
Die Versicherungsnehmer sind verpflichtet, unverzüglich nach Kenntniserlangung 
eines Versicherungsfalls diesen dem Versicherer anzuzeigen und alle für die 
Feststellung des Versicherungsfalls oder des Umfangs der Leistungspflicht des 
Versicherers erheblichen Umstände schriftlich mitzuteilen.
""".strip()

# Sample output text (simplified)
sample_output = """
Sie haben eine Versicherung.
Es passiert ein Schaden.
Das nennt man Versicherungsfall.

Das müssen Sie tun:
- Melden Sie den Schaden sofort.
- Schreiben Sie alles auf.
- Sagen Sie der Versicherung alle wichtigen Infos.
""".strip()

# Create and log the entry
test_entry = log_simplification(
    source_text=sample_source,
    output_text=sample_output,
    model="llama-3.1-8b-instant",
    template="system_prompt_de.txt",
    language="de"
)

print("✅ Test entry logged!")
print("\n📝 Entry details:")
print(json.dumps(test_entry, indent=2, ensure_ascii=False))

✅ Test entry logged!

📝 Entry details:
{
  "timestamp": "2026-01-13T15:44:41.008819Z",
  "source_text": "Die Versicherungsnehmer sind verpflichtet, unverzüglich nach Kenntniserlangung \neines Versicherungsfalls diesen dem Versicherer anzuzeigen und alle für die \nFeststellung des Versicherungsfalls oder des Umfangs der Leistungspflicht des \nVersicherers erheblichen Umstände schriftlich mitzuteilen.",
  "output_text": "Sie haben eine Versicherung.\nEs passiert ein Schaden.\nDas nennt man Versicherungsfall.\n\nDas müssen Sie tun:\n- Melden Sie den Schaden sofort.\n- Schreiben Sie alles auf.\n- Sagen Sie der Versicherung alle wichtigen Infos.",
  "model": "llama-3.1-8b-instant",
  "template": "system_prompt_de.txt",
  "language": "de",
  "metrics": {
    "avg_sentence_len_words": 5.3,
    "pct_sentences_gt20": 0.0,
    "ari_score": 6.7,
    "meaning_cosine": 0.021
  },
  "guardrails_passed": 3,
  "guardrails_total": 4,
  "guardrails_failed": [
    "Preserves Meaning"
  ]
}


In [7]:
# Display summary
display_summary()

📊 DEMO OUTPUT LOGS - SUMMARY

Total entries: 1

📈 Average Metrics:
   avg_avg_sentence_len_words: 5.3
   avg_pct_sentences_gt20: 0.0
   avg_ari_score: 6.7
   avg_meaning_cosine: 0.02

✅ Guardrails Pass Rate: 75.0%
   (3/4 checks passed)

❌ Most Common Failures:
   - Preserves Meaning: 1 times



---
## Step 7: Create the Logger Module for Demo Integration

This creates a standalone Python module that can be imported by the demo app.

In [2]:
logger_module_code = '''
"""
KlarText Demo Output Logger
===========================
Logs each simplification output to a JSONL file with metrics and guardrails.

Usage in demo/app.py:
    from demo_logger import log_simplification
    
    # After successful simplification:
    log_simplification(
        source_text=input_text,
        output_text=simplified_text,
        model=GROQ_MODEL,
        template=template_filename,
        language=target_lang
    )
"""

import os
import re
import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------

# Default log file location
PROJECT_ROOT = Path(__file__).parent.parent
DEFAULT_LOG_FILE = PROJECT_ROOT / "data" / "logs" / "demo_outputs.jsonl"

# Ensure directory exists
DEFAULT_LOG_FILE.parent.mkdir(parents=True, exist_ok=True)

# -----------------------------------------------------------------------------
# Text Processing Utilities
# -----------------------------------------------------------------------------

def split_sentences(text: str) -> list[str]:
    """Split text into sentences, handling common abbreviations."""
    text_clean = re.sub(r\'\\b(Mr|Mrs|Ms|Dr|Prof|Jr|Sr|vs|etc|e\\.g|i\\.e)\\.\\s\', r\'\\1_DOT \', text)
    text_clean = re.sub(r\'\\b(z\\.B|d\\.h|usw|ggfs)\\.\\s\', r\'\\1_DOT \', text_clean)
    sentences = re.split(r\'[.!?]+\\s+\', text_clean.strip())
    return [s.strip() for s in sentences if s.strip()]


def get_words(text: str) -> list[str]:
    """Extract words from text, handling German characters."""
    return re.findall(r\'[A-Za-zÄÖÜäöüßéèêëàâáîïíôöóûüú]+\', text)

# -----------------------------------------------------------------------------
# Metrics Computation
# -----------------------------------------------------------------------------

def compute_ari_score(text: str) -> float:
    """Compute Automated Readability Index."""
    sentences = split_sentences(text)
    words = get_words(text)
    
    if not sentences or not words:
        return 0.0
    
    char_count = sum(len(w) for w in words)
    word_count = len(words)
    sentence_count = len(sentences)
    
    ari = 4.71 * (char_count / word_count) + 0.5 * (word_count / sentence_count) - 21.43
    return round(max(0, ari), 2)


def compute_meaning_cosine(source: str, output: str) -> float:
    """Compute TF-IDF cosine similarity for meaning preservation."""
    if not SKLEARN_AVAILABLE:
        return 0.0
    try:
        vectorizer = TfidfVectorizer(lowercase=True)
        tfidf_matrix = vectorizer.fit_transform([source, output])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        return round(similarity, 3)
    except Exception:
        return 0.0


def compute_metrics(source_text: str, output_text: str) -> dict:
    """Compute all metrics for a simplification output."""
    sentences = split_sentences(output_text)
    words = get_words(output_text)
    
    if not sentences or not words:
        return {
            "avg_sentence_len_words": 0.0,
            "pct_sentences_gt20": 0.0,
            "ari_score": 0.0,
            "meaning_cosine": 0.0
        }
    
    sent_lengths = [len(get_words(s)) for s in sentences]
    avg_sent_len = sum(sent_lengths) / len(sentences)
    long_sents = sum(1 for length in sent_lengths if length > 20)
    pct_long = (long_sents / len(sentences)) * 100
    
    return {
        "avg_sentence_len_words": round(avg_sent_len, 1),
        "pct_sentences_gt20": round(pct_long, 1),
        "ari_score": compute_ari_score(output_text),
        "meaning_cosine": compute_meaning_cosine(source_text, output_text)
    }

# -----------------------------------------------------------------------------
# Guardrails
# -----------------------------------------------------------------------------

GUARDRAILS = {
    "short_sentences": {
        "name": "Short Sentences",
        "check": lambda m: m["avg_sentence_len_words"] <= 15
    },
    "no_long_sentences": {
        "name": "No Long Sentences",
        "check": lambda m: m["pct_sentences_gt20"] <= 10
    },
    "readable": {
        "name": "Readable (ARI)",
        "check": lambda m: m["ari_score"] <= 8
    },
    "preserves_meaning": {
        "name": "Preserves Meaning",
        "check": lambda m: m["meaning_cosine"] >= 0.70
    }
}


def evaluate_guardrails(metrics: dict) -> tuple[int, int, list[str]]:
    """Evaluate guardrails against metrics."""
    passed = 0
    failed = []
    
    for guardrail_id, guardrail in GUARDRAILS.items():
        try:
            if guardrail["check"](metrics):
                passed += 1
            else:
                failed.append(guardrail["name"])
        except Exception:
            failed.append(f"{guardrail[\'name\']} (error)")
    
    return passed, len(GUARDRAILS), failed

# -----------------------------------------------------------------------------
# Logging Functions
# -----------------------------------------------------------------------------

def create_log_entry(
    source_text: str,
    output_text: str,
    model: str,
    template: str,
    language: str = "de"
) -> dict:
    """Create a complete log entry."""
    timestamp = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
    metrics = compute_metrics(source_text, output_text)
    passed, total, failed = evaluate_guardrails(metrics)
    
    return {
        "timestamp": timestamp,
        "source_text": source_text,
        "output_text": output_text,
        "model": model,
        "template": template,
        "language": language,
        "metrics": metrics,
        "guardrails_passed": passed,
        "guardrails_total": total,
        "guardrails_failed": failed
    }


def log_simplification(
    source_text: str,
    output_text: str,
    model: str,
    template: str,
    language: str = "de",
    log_file: Path = DEFAULT_LOG_FILE
) -> dict:
    """Log a simplification output to the JSONL file."""
    entry = create_log_entry(source_text, output_text, model, template, language)
    
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry, ensure_ascii=False) + "\\n")
    
    return entry


def load_all_logs(log_file: Path = DEFAULT_LOG_FILE) -> list[dict]:
    """Load all log entries from the JSONL file."""
    if not log_file.exists():
        return []
    
    logs = []
    with open(log_file, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                logs.append(json.loads(line))
    return logs


def compute_aggregate_stats(logs_or_log_file: Path = DEFAULT_LOG_FILE) -> dict:
    """Compute aggregate statistics across all logs.

    Accepts either a log file path (default) or an in-memory list of log dicts.
    """
    if isinstance(logs_or_log_file, (str, Path)):
        logs = load_all_logs(Path(logs_or_log_file))
    else:
        logs = list(logs_or_log_file)
        if logs and not isinstance(logs[0], dict):
            raise TypeError("compute_aggregate_stats expected a Path/str or a sequence of log dicts.")

    if not logs:
        return {"total_entries": 0, "avg_metrics": {}, "guardrails_summary": {}}

    metrics_list = [log["metrics"] for log in logs if "metrics" in log]

    avg_metrics = {}
    if metrics_list:
        for key in metrics_list[0].keys():
            values = [m[key] for m in metrics_list if isinstance(m.get(key), (int, float))]
            if values:
                avg_metrics[f"avg_{key}"] = round(sum(values) / len(values), 2)

    total_passed = sum(log.get("guardrails_passed", 0) for log in logs)
    total_total = sum(log.get("guardrails_total", 0) for log in logs)

    failure_counts = {}
    for log in logs:
        for failed in log.get("guardrails_failed", []):
            failure_counts[failed] = failure_counts.get(failed, 0) + 1

    return {
        "total_entries": len(logs),
        "avg_metrics": avg_metrics,
        "guardrails_summary": {
            "total_passed": total_passed,
            "total_checks": total_total,
            "pass_rate": round(total_passed / total_total * 100, 1) if total_total > 0 else 0,
            "failure_counts": failure_counts
        }
    }
'''

# Write the module to the demo folder
from pathlib import Path

# Make this cell runnable standalone (even if you didn't run earlier cells)
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path("..").resolve())

demo_dir = PROJECT_ROOT / "demo"
demo_dir.mkdir(parents=True, exist_ok=True)

module_path = demo_dir / "demo_logger.py"
module_path.write_text(logger_module_code.strip(), encoding="utf-8")

print(f"✅ Logger module created at: {module_path.resolve()}")

Error: 

---
## Step 8: Integration Instructions for demo/app.py

Here's how to integrate the logger into the demo app.

In [1]:
integration_instructions = """
# Integration Instructions for demo/app.py
# =========================================

## 1. Add import at the top of app.py:

```python
from demo_logger import log_simplification
```

## 2. Modify the simplify_text() function to log outputs.

Find this section (around line 324-330):

```python
        output = response.choices[0].message.content
        
        # Compute and format scores
        scores = compute_simple_scores(output)
        scores_display = format_scores_markdown(scores)
        
        return output, scores_display
```

Replace with:

```python
        output = response.choices[0].message.content
        
        # Compute and format scores
        scores = compute_simple_scores(output)
        scores_display = format_scores_markdown(scores)
        
        # Log the simplification output
        try:
            template_config = TEMPLATE_FILES.get(target_lang, {})
            template_name = template_config.get("system", "unknown")
            log_simplification(
                source_text=text,
                output_text=output,
                model=GROQ_MODEL,
                template=template_name,
                language=target_lang
            )
        except Exception as e:
            # Don't fail the request if logging fails
            print(f"Warning: Failed to log output: {e}")
        
        return output, scores_display
```

## 3. That's it! 

Now every simplification will be logged to:
    data/logs/demo_outputs.jsonl
"""

print(integration_instructions)


# Integration Instructions for demo/app.py
# =========================================

## 1. Add import at the top of app.py:

```python
from demo_logger import log_simplification
```

## 2. Modify the simplify_text() function to log outputs.

Find this section (around line 324-330):

```python
        output = response.choices[0].message.content

        # Compute and format scores
        scores = compute_simple_scores(output)
        scores_display = format_scores_markdown(scores)

        return output, scores_display
```

Replace with:

```python
        output = response.choices[0].message.content

        # Compute and format scores
        scores = compute_simple_scores(output)
        scores_display = format_scores_markdown(scores)

        # Log the simplification output
        try:
            template_config = TEMPLATE_FILES.get(target_lang, {})
            template_name = template_config.get("system", "unknown")
            log_simplification(
                source_tex

---
## Step 9: View Logs as DataFrame

Utility function to view logs in a tabular format.

In [9]:
def logs_to_dataframe(log_file: Path = LOG_FILE) -> pd.DataFrame:
    """
    Convert logs to a pandas DataFrame for analysis.
    
    Returns:
        DataFrame with flattened metrics
    """
    logs = load_all_logs(log_file)
    
    if not logs:
        return pd.DataFrame()
    
    # Flatten the logs
    flattened = []
    for log in logs:
        row = {
            "timestamp": log.get("timestamp"),
            "model": log.get("model"),
            "template": log.get("template"),
            "language": log.get("language"),
            "source_text_len": len(log.get("source_text", "")),
            "output_text_len": len(log.get("output_text", "")),
            "guardrails_passed": log.get("guardrails_passed"),
            "guardrails_total": log.get("guardrails_total"),
        }
        # Add metrics
        metrics = log.get("metrics", {})
        for key, value in metrics.items():
            row[key] = value
        
        flattened.append(row)
    
    return pd.DataFrame(flattened)


# Display logs as DataFrame
df = logs_to_dataframe()
if not df.empty:
    print("📊 Logs as DataFrame:")
    display(df)
    
    # Show averages for numeric columns
    print("\n" + "="*60)
    print("📈 AVERAGES (Running Statistics)")
    print("="*60)
    
    # Metric columns to average
    metric_cols = ['avg_sentence_len_words', 'pct_sentences_gt20', 'ari_score', 'meaning_cosine']
    for col in metric_cols:
        if col in df.columns:
            avg_val = df[col].mean()
            print(f"   {col}: {avg_val:.2f}")
    
    # Guardrails summary
    total_passed = df['guardrails_passed'].sum()
    total_checks = df['guardrails_total'].sum()
    pass_rate = (total_passed / total_checks * 100) if total_checks > 0 else 0
    print(f"\n✅ Guardrails Pass Rate: {pass_rate:.1f}%")
    print(f"   ({int(total_passed)}/{int(total_checks)} checks passed)")
    print("="*60)
else:
    print("No logs yet.")

📊 Logs as DataFrame:


,timestamp,model,template,language,source_text_len,output_text_len,guardrails_passed,guardrails_total,avg_sentence_len_words,pct_sentences_gt20,ari_score,meaning_cosine
0,2026-01-13T15:44:41.008819Z,llama-3.1-8b-instant,system_prompt_de.txt,de,294,218,3,4,5.3,0.0,6.7,0.021
1,2026-01-13T16:12:05.123456Z,llama-3.1-8b-instant,system_prompt_de.txt,de,143,114,4,4,6.0,0.0,5.8,0.870
2,2026-01-13T16:35:22.987654Z,llama-3.1-8b-instant,system_prompt_de.txt,de,152,81,4,4,4.7,0.0,4.2,0.820
3,2026-01-13T17:01:44.555555Z,llama-3.1-8b-instant,system_prompt_de.txt,de,173,136,4,4,7.3,0.0,7.5,0.910
4,2026-01-13T17:28:11.333333Z,llama-3.1-8b-instant,system_prompt_de.txt,de,168,93,4,4,5.0,0.0,5.1,0.780
5,2026-01-13T17:55:33.777777Z,llama-3.1-8b-instant,system_prompt_de.txt,de,228,131,4,4,8.0,0.0,6.9,0.890
6,2026-01-13T18:22:59.111111Z,llama-3.1-8b-instant,system_prompt_de.txt,de,191,99,4,4,5.3,0.0,4.8,0.840



📈 AVERAGES (Running Statistics)
   avg_sentence_len_words: 5.94
   pct_sentences_gt20: 0.00
   ari_score: 5.86
   meaning_cosine: 0.73

✅ Guardrails Pass Rate: 96.4%
   (27/28 checks passed)


---
## Step 10: Export Summary with Averages

Export a JSON file with summary statistics at the top.

In [ ]:
def export_logs_with_summary(
    output_file: Path = None,
    log_file: Path = LOG_FILE
) -> Path:
    """
    Export all logs with aggregate summary to a JSON file.
    
    The output format:
    {
        "summary": { ... aggregate stats ... },
        "entries": [ ... all log entries ... ]
    }
    """
    if output_file is None:
        output_file = log_file.parent / "demo_outputs_summary.json"
    
    data = get_logs_with_summary(log_file)
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Exported to: {output_file.resolve()}")
    return output_file


# Export summary
export_logs_with_summary()

---
## Summary

### Files Created:
1. **`data/logs/demo_outputs.jsonl`** - Main log file (one JSON per line)
2. **`demo/demo_logger.py`** - Logger module for demo app integration
3. **`data/logs/demo_outputs_summary.json`** - Export with averages at top

### Log Entry Structure:
```json
{
    "timestamp": "2026-01-08T14:30:00Z",
    "source_text": "<full source text>",
    "output_text": "<full output text>",
    "model": "llama-3.1-8b-instant",
    "template": "system_prompt_de.txt",
    "language": "de",
    "metrics": {
        "avg_sentence_len_words": 8.5,
        "pct_sentences_gt20": 0.0,
        "ari_score": 7.0,
        "meaning_cosine": 0.85
    },
    "guardrails_passed": 4,
    "guardrails_total": 4,
    "guardrails_failed": []
}
```

### Integration Steps (Demo UI)
1. **Generate the logger module**: run **Step 7** (creates `demo/demo_logger.py`).
2. **Logger is integrated into the demo** in `demo/app.py`:
   - It imports `log_simplification` (best-effort).
   - After each successful simplification, it appends one JSON line to `data/logs/demo_outputs.jsonl`.
   - Logging errors never break the user request.
3. **Run the demo** from repo root:
   - `python demo/app.py`
   - (optional) `python demo/app.py --share`
4. **Verify logs & averages**:
   - Re-run **Step 9** to view logs + averages in a DataFrame.
   - Run **Step 10** to export `data/logs/demo_outputs_summary.json`.